In [ ]:
import cv2 as cv2
import numpy as np

def obtainFCol0(img,rows,cols):
    result=np.zeros((rows,cols),dtype=int)
    for i in range (0,rows,1):
        for j in range (0,cols,1):
            if(j==0):
                result[i][j]=0
            else:
                result[i][j]=img[i][j-1]
    return result

def obtainFR0(img,rows,cols):
    result=np.array([0]*img)
    for i in range (0,rows,1):
        for j in range (0,cols,1):
            if(i==0):
                result[i][j]=0
            else:
                result[i][j]=img[i-1][j]
    return result

def IxIymul(arr1,arr2,rows,cols):
    result=np.array([0]*rows,[0]*cols)
    for i in range(0,rows,1):
        for j in range(0, cols,1):
            result[i][j]=arr1[i][j]*arr2[i][j]
    return result

def elementWiseMul(arr1,arr2,rows,cols):
    result=np.zeros((rows,cols),dtype=int)
    for i in range(0,rows,1):
        for j in range(0,cols,1):
            result[i][j]=np.multiply(arr1[i][j],arr2[i][j])
    return result

def MeasureCArray(Ixsq,Iysq,IxIy,rows,cols,win_size):
    R=0
    C=0
    margin=int(np.floor(np.divide(win_size,2)))
    margin=2*margin
    M=np.zeros((2,2),dtype=int)
    k=0
    # delete right left top bottom row
    C_arr=np.zeros((rows-margin)*(cols-margin))
    while R<=rows-win_size:
        while C<=cols-win_size:
            for i in range(R,R+win_size-1,1):
                for j in range(C,C+win_size-1,1):
                    M[0][0]=M[0][0]+Ixsq[i][j]
                    M[0][1]=M[0][1]+IxIy[i][j]
                    M[1][0]=M[1][0]+IxIy[i][j]
                    M[1][1]=M[1][1]+Iysq[i][j]
            M=np.divide(M,np.power(win_size,2))
            detM=np.linalg.det(M)
            trace=np.trace(M)
            alpha=0.04
            C_arr[k]=np.subtract(detM,np.multiply(alpha,np.power(trace,2)))
            k=k+1
            M=np.zeros((2,2),dtype=int)
            C=C+1
        C=0
        R=R+1
    return C_arr


    
def thresholdCMatrix(givenMatrix,limit):
    size=givenMatrix.shape
    rows=size[0]
    cols=size[1]
    result=np.zeros((rows,cols))
    for i in range (0,rows,1):
        for j in range (0,cols,1):
            if (givenMatrix[i][j]<limit):
                result[i][j]=0
            else:
                result[i][j]=1
    return result
                
    
def findLocalMinCMatrix(myMat,win_size):
    R=0
    C=0
    result_mat=myMat
    maxI=0
    maxJ=0
    size=myMat.shape
    rows=size[0]
    cols=size[1]
    to_fwd=np.zeros((rows,cols))
    while R<=rows-win_size+1:
        while C<=cols-win_size+1:
            maxI=R
            maxJ=C
            maxVal=myMat[maxI][maxJ]
            for i in range(R,R+win_size-1,1):
                for j in range(C,C+win_size-1,1):
                    if(maxVal<myMat[i][j]):
                        result_mat[maxI][maxJ]=0
                        maxVal=myMat[i][j]
                        maxI=i
                        maxJ=j
                    else:
                        result_mat[i][j]=0
            to_fwd[maxI][maxJ]=result_mat[maxI][maxJ]
            result_mat[maxI][maxJ]=1
            C=C+1
        C=0
        R=R+1
    return result_mat,to_fwd


def makeRedDots(matRef,matR):
    size=matRef.shape
    rows=size[0]
    cols=size[1]
    result=matR
    for i in range (0,rows,1):
        for j in range (0,cols,1):
            if(matRef[i][j]==1):
                result[i][j][0]=0
                result[i][j][1]=255
                result[i][j][2]=0

    return result
                
    
filename='C:/EME/8th sem/CV/assignment/#1/supreme_court.jpg'
img=cv2.imread(filename)
grayImg=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#window for supreme court 7 and threshold 20000
#window for harry potter =7 and threshold 25000
win_size=5
cv2.imshow('gray_orig',grayImg)
cv2.waitKey(0)


size=grayImg.shape
rows=size[0]
cols=size[1]
FC0=obtainFCol0(grayImg, rows, cols)
FR0=obtainFR0(grayImg, rows, cols)  

Ix=np.subtract(grayImg,FC0)
Iy=np.subtract(grayImg,FR0)
IxIy=elementWiseMul(Ix, Iy, rows, cols)
Ixsq=elementWiseMul(Ix, Ix, rows, cols)
Iysq=elementWiseMul(Iy, Iy, rows, cols)

C_Array=MeasureCArray(Ixsq, Iysq, IxIy, rows, cols, win_size)
margin=int(np.floor(np.divide(win_size,2)))
margin=2*margin
C_matrix=np.reshape(C_Array,(rows-margin,cols-margin))
C_mat_THolded=thresholdCMatrix(C_matrix, 20000)

cv2.imshow('results-thresholded',C_mat_THolded)
cv2.waitKey(0)

C_mat_local_min,fwd_matrix=findLocalMinCMatrix(C_matrix, win_size)
cv2.imshow('results-with-local-min',C_mat_local_min)
cv2.waitKey(0)

Final_dotted_ver=makeRedDots(C_mat_local_min,img)
cv2.imshow('final-dotted-overlapped',Final_dotted_ver)
cv2.waitKey(0)

